In [ ]:
%reload_ext autoreload
%autoreload 2


from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from PIL import Image

# Load the image
image_path = "crepe_data/VG_100K_2/1.jpg"
# image = mpimg.imread(image_path)
image = Image.open(image_path)
image = image.convert('RGB')

# plt.title("Sheep Image")
# plt.xlabel("X pixel scaling")
# plt.ylabel("Y pixel scaling")
plt.imshow(image)
plt.show()

target_image_id=15

In [ ]:
img_caption_file_name="crepe_data/prod_hard_negatives/prod_vg_hard_negs_swap_all.csv"

import pandas as pd
 
caption_pd = pd.read_csv(img_caption_file_name)

In [ ]:
print(caption_pd.iloc[4]['caption'], caption_pd.iloc[4]['image_id'])
print(caption_pd.iloc[3]['caption'], caption_pd.iloc[3]['image_id'])

In [ ]:
len(caption_pd)

In [ ]:
from image_utils import load_crepe_datasets, load_other_crepe_images
import os

import IPython

data_path="/data2/wuyinjun/"
query_path=os.getcwd() #curr_dir = str(globals()["_dh"][0])
dataset_name="crepe"
full_data_path = os.path.join(data_path, dataset_name)
# raw_img_ls = [image]
queries, raw_img_ls, sub_queries_ls, img_idx_ls = load_crepe_datasets(full_data_path, query_path)
img_idx_ls, raw_img_ls = load_other_crepe_images(full_data_path, query_path, img_idx_ls, raw_img_ls, total_count=100)

In [ ]:
print(len(raw_img_ls))

In [ ]:
import torch
from transformers import CLIPModel, AutoProcessor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# processor = ViTImageProcessor.from_pretrained('google/vit-large-patch16-224')
# model = ViTForImageClassification.from_pretrained('google/vit-large-patch16-224').to(device)
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)
# processor = AutoProcessor.from_pretrained("openai/clip-vit-large-patch14")
raw_processor = AutoProcessor.from_pretrained("openai/clip-vit-large-patch14")
# processor =  lambda images: raw_processor(images=images, return_tensors="pt", padding=False, do_resize=False, do_center_crop=False)["pixel_values"]
processor =  lambda images: raw_processor(images=images, return_tensors="pt")["pixel_values"]
text_processor =  lambda text: raw_processor(text=[text], return_tensors="pt", padding=True, truncation=True)
img_processor =  lambda images: raw_processor(images=images, return_tensors="pt")["pixel_values"]
model = model.eval()

In [ ]:
from image_utils import *

new_raw_img_ls = raw_img_ls #[raw_img_ls[idx] for idx in range(1)]

cl = ConceptLearner(new_raw_img_ls, model, vit_forward, processor, dataset_name, device)

patch_count=4

img_emb, patch_emb, masks, bboxes, img_per_patch = cl.get_patches(images=new_raw_img_ls, n_patches=patch_count, method="slic", compute_img_emb=True)

# img_emb, patch_emb_ls, masks_ls, bboxes_ls, img_per_patch_ls = convert_samples_to_concepts(args, model, raw_img_ls, processor, device, patch_count_ls=patch_count_ls)




In [ ]:
import matplotlib.pyplot as plt
import cv2
import PIL
import ipyplot

from tqdm import tqdm
target_image_id  =15
print("target_image_id::", target_image_id)
image_id = target_image_id

full_masked_image_ls = []

for image_id in tqdm(range(len(raw_img_ls))):
# for image_id in tqdm(range(1)):

    first_mask = masks[image_id]

    # print(len(first_mask))

    first_img = raw_img_ls[image_id]

    first_bbox_ls = bboxes[image_id]

    # print(len(first_bbox_ls))

    masked_image_ls = []

    for idx in range(len(first_bbox_ls)):
        first_bbox = first_bbox_ls[idx]
        masked_image = np.copy(first_img)
        # # print(masked_image.shape)
        masked_image[first_mask != (idx + 1)] = 255 # Set pixels outside the mask to 0
        # curr_patch = PIL.Image.new('RGB', first_img.size)
        # curr_patch.paste(first_img.copy().crop(first_bbox), box=first_bbox)
        # # masked_image = PIL.ImageOps.pad(curr_patch.crop(first_bbox), (224, 224))
        # masked_image = curr_patch.crop(first_bbox)
        masked_image_ls.append(masked_image)

    full_masked_image_ls.append(masked_image_ls)

ipyplot.plot_images(full_masked_image_ls[target_image_id], max_images=len(bboxes[target_image_id]))   

# masked_image = cv2.rectangle(Image.fromarray(masked_image), (first_bbox[0], first_bbox[1]), (first_bbox[2], first_bbox[3]), (255, 0, 0), 2)

# Display the masked part of the image
# plt.imshow(masked_image)
# plt.axis('off')
# plt.show()

# print(np.unique(first_mask))

In [ ]:
first_mask = masks[0]
first_img = raw_img_ls[0]
first_bbox_ls = bboxes[0]
print(np.unique(first_mask))

first_img_cp = first_img.copy()
first_img_cp_array = np.array(first_img_cp)
first_img_cp_array[first_mask!=2] = 0

plt.imshow(Image.fromarray(first_img_cp_array))

plt.axis('off')
plt.show()

In [ ]:
further_masks = get_slic_segments([Image.fromarray(first_img_cp_array)], n_segments=8)

In [ ]:
print(np.unique(further_masks))

In [ ]:
first_img_cp_array_cp1 = np.copy(first_img_cp_array)
first_img_cp_array_cp1[further_masks[0] != 5] = 100
plt.imshow(Image.fromarray(first_img_cp_array_cp1))

plt.axis('off')
plt.show()

In [ ]:
sub_idx_ls = np.nonzero(np.array(img_per_patch) == target_image_id)[0].tolist()
first_patch_emb = np.stack([patch_emb[idx] for idx in sub_idx_ls])
print(first_patch_emb.shape)
print(len(bboxes[0]))

In [ ]:
# query="the street has a parking meter and a lamp post"
# query="sign on a building with a window and balcony"
# query="sign on a building with a window and balcony"#. 
query="the door has a frame around it, and there is a second window on the building."
inputs = text_processor(query)
inputs = {key: val.to(device) for key, val in inputs.items()}
text_features = model.get_text_features(**inputs)
print(text_features.shape)

# segment images with slic

In [ ]:
all_patch_cos_sim_ls = []
all_img_cos_sim_ls = []

for image_id in tqdm(range(len(raw_img_ls))):

    sub_idx_ls = np.nonzero(np.array(img_per_patch) == image_id)[0].tolist()
    first_patch_emb = np.stack([patch_emb[idx] for idx in sub_idx_ls])

    # first_bbox_ls = bboxes[image_id]
    patch_cos_sim_ls = []
    for idx in range(len(first_patch_emb)):
        cosin_sim1 = torch.nn.functional.cosine_similarity(text_features, torch.from_numpy(first_patch_emb[idx]).view(1,-1).to(device), dim=-1)
        patch_cos_sim_ls.append(cosin_sim1)
        # print(idx, cosin_sim1)
    cosin_sim2 = torch.nn.functional.cosine_similarity(text_features, img_emb[image_id].view(1,-1).to(device), dim=-1)
    all_img_cos_sim_ls.append(cosin_sim2)
    all_patch_cos_sim_ls.append(torch.tensor(patch_cos_sim_ls))

# print(cosin_sim1, cosin_sim2)
all_img_cos_sim_tensor = torch.tensor(all_img_cos_sim_ls)
# all_patch_cos_sim_tensor = torch.tensor(all_patch_cos_sim_ls)

In [ ]:
print(all_img_cos_sim_tensor.argmax())
print(all_img_cos_sim_tensor.max())
all_patch_cos_sim_max_scores = torch.tensor([torch.max(all_patch_cos_sim_ls[idx]) for idx in range(len(all_patch_cos_sim_ls))])
print(torch.argmax(all_patch_cos_sim_max_scores))
print(torch.max(all_patch_cos_sim_max_scores))

In [ ]:
full_img_sorted_ids = torch.sort(all_img_cos_sim_tensor, descending=True)[1]
print(torch.nonzero(full_img_sorted_ids == target_image_id))

In [ ]:
all_patch_cos_sim_max_scores_full = torch.stack([all_patch_cos_sim_max_scores.view(-1), all_img_cos_sim_tensor.view(-1)], dim=1)
all_patch_cos_sim_max_scores_full_final = torch.max(all_patch_cos_sim_max_scores_full, dim=1)[0]
print(all_patch_cos_sim_max_scores_full_final)
sorted_ids = torch.sort(all_patch_cos_sim_max_scores_full_final, descending=True)[1]

In [ ]:
print(sorted_ids)
print(all_patch_cos_sim_max_scores_full_final[target_image_id])
print(torch.nonzero(sorted_ids == target_image_id))

In [ ]:
print(torch.max(all_patch_cos_sim_ls[target_image_id]), torch.argmax(all_patch_cos_sim_ls[target_image_id]))
print(torch.sort(all_patch_cos_sim_ls[target_image_id], descending=True))

In [ ]:
image = raw_img_ls[target_image_id].convert('RGB')

# plt.title("Sheep Image")
# plt.xlabel("X pixel scaling")
# plt.ylabel("Y pixel scaling")
plt.imshow(image)
plt.show()

In [ ]:
print(np.unique(first_mask))
print(np.sum(first_mask == 15), np.sum(first_mask == 4))
compound_mask = np.logical_or(first_mask == 15, first_mask == 4)

print(np.sum(compound_mask))

y_ls, x_ls = np.nonzero(compound_mask)

x1, y1, x2, y2 = np.min(x_ls), np.min(y_ls), np.max(x_ls), np.max(y_ls)
compound_box = [x1, y1, x2, y2]
print(compound_box)


curr_patch = PIL.Image.new('RGB', first_img.size)
curr_patch.paste(first_img.copy().crop(compound_box), box=compound_box)
masked_image = PIL.ImageOps.pad(curr_patch.crop(compound_box), (224, 224))

plt.figure(figsize=(20,20))

plt.imshow(masked_image)

plt.axis('off')
plt.show()

In [ ]:

masked_image_emb, masked_img_for_patch = get_patches_from_bboxes(cl.input_to_latent, cl.model, [first_img], [[compound_box]], cl.input_processor, device=cl.device, resize=cl.image_size)
print(masked_image_emb.shape)

In [ ]:
cosin_sim3 = torch.nn.functional.cosine_similarity(text_features, masked_image_emb.view(1,-1).to(device), dim=-1)
print(cosin_sim3)

# segment images with sam

In [ ]:
from segment_anything import SamPredictor, sam_model_registry
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor


def segment_one_image(image):
    checkpoint_path="/data2/wuyinjun/sam/sam_vit_h_4b8939.pth"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    sam = sam_model_registry["vit_h"](checkpoint=checkpoint_path)
    sam.to(device=device)
    mask_generator = SamAutomaticMaskGenerator(sam
    , points_per_side=16)
    masks = mask_generator.generate(image)

    # print(len(masks))
    return masks


def segment_image_ls(image_ls):
    checkpoint_path="/data2/wuyinjun/sam/sam_vit_h_4b8939.pth"
    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

    sam = sam_model_registry["vit_h"](checkpoint=checkpoint_path)
    sam.to(device=device)
    mask_generator = SamAutomaticMaskGenerator(sam , points_per_side=16)
    masks_ls = []
    for image in tqdm(image_ls):
        masks = mask_generator.generate(np.array(image))
        masks_ls.append(masks)

    # print(len(masks))
    return masks_ls

# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# masks_ls = []
# for idx in tqdm(range(len(raw_img_ls))):
#     masks = segment_one_image(np.array(raw_img_ls[idx]))
#     masks_ls.append(masks)
masks_ls = segment_image_ls(raw_img_ls)
# predictor = SamPredictor(sam)
# predictor.set_image(raw_img_ls[0])
# masks, _, _ = predictor.predict()

In [ ]:
utils.save(masks_ls, "output/crepe_masks.pkl")
# utils.dump_pickle("crepe_masks.pkl", masks_ls)

In [ ]:
masks_ls = utils.load("output/crepe_masks.pkl")


In [ ]:


full_masked_image_emb_ls = []
print("total image count::", len(raw_img_ls))
for idx in range(len(raw_img_ls)):
    print("image idx::", idx)
    masks = masks_ls[idx]
    masked_image_ls = []
    for mask in masks:
        # print(mask.shape)
        # print(np.unique(mask['segmentation']))
        y_ls, x_ls = np.nonzero(mask['segmentation'])
        bbox = [np.min(x_ls), np.min(y_ls), np.max(x_ls), np.max(y_ls)]
        curr_patch = PIL.Image.new('RGB', raw_img_ls[idx].size)
        curr_patch.paste(raw_img_ls[idx].copy().crop(bbox), box=bbox)
        # masked_image0 = curr_patch # PIL.ImageOps.pad(curr_patch.crop(bbox), (224, 224))
        # masked_image = PIL.ImageOps.pad(curr_patch.crop(bbox), curr_patch.size) 
        masked_image = curr_patch.crop(bbox).convert('RGB')
        if masked_image.size[0] > 1 and masked_image.size[1] > 1:
            masked_image_ls.append(masked_image)
        else:
            if masked_image.size[0] == 1:
                masked_image_ls.append(masked_image.resize((2, masked_image.size[1])))
            else:
                masked_image_ls.append(masked_image.resize((masked_image.size[0], 2)))
    
    # print(masked_image_ls[63].size)
    # print(masked_image_ls[62].size)
    masked_image_emb_tensor = get_image_embeddings(masked_image_ls, cl.input_processor, cl.input_to_latent, cl.model, not_normalize=False)
    
    # masked_image_emb_ls.append(masked_image_emb.view(-1))
        
    # masked_image_emb_tensor = torch.stack(masked_image_emb_ls)
    
    full_masked_image_emb_ls.append(masked_image_emb_tensor)
    


In [ ]:
utils.save(full_masked_image_emb_ls, "output/crepe_masked_image_emb_ls.pkl")

In [ ]:
full_masked_image_emb_ls = utils.load("output/crepe_masked_image_emb_ls.pkl")

In [ ]:
all_patch_cos_sim_ls2 = []
# all_img_cos_sim_ls = []

for image_id in tqdm(range(len(raw_img_ls))):

    sub_idx_ls = np.nonzero(np.array(img_per_patch) == image_id)[0].tolist()
    # first_patch_emb = np.stack([patch_emb[idx] for idx in sub_idx_ls])
    masked_image_emb_ls = full_masked_image_emb_ls[image_id]

    # first_bbox_ls = bboxes[image_id]
    patch_cos_sim_ls = []
    for idx in range(len(masked_image_emb_ls)):
        cosin_sim1 = torch.nn.functional.cosine_similarity(text_features, masked_image_emb_ls[idx].view(1,-1).to(device), dim=-1)
        patch_cos_sim_ls.append(cosin_sim1)
        # print(idx, cosin_sim1)
    # cosin_sim2 = torch.nn.functional.cosine_similarity(text_features, img_emb[image_id].view(1,-1).to(device), dim=-1)
    # all_img_cos_sim_ls.append(cosin_sim2)
    all_patch_cos_sim_ls2.append(torch.tensor(patch_cos_sim_ls))

# print(cosin_sim1, cosin_sim2)
# all_img_cos_sim_tensor = torch.tensor(all_img_cos_sim_ls)

In [ ]:
all_patch_cos_sim_max_scores2 = torch.tensor([torch.max(all_patch_cos_sim_ls2[idx]) for idx in range(len(all_patch_cos_sim_ls2))])
print(all_patch_cos_sim_max_scores2[15])
print(torch.argmax(all_patch_cos_sim_max_scores2))
print(torch.max(all_patch_cos_sim_max_scores2))

full_img_sorted_ids = torch.sort(all_patch_cos_sim_max_scores2, descending=True)[1]
print(torch.nonzero(full_img_sorted_ids == 0))

In [ ]:
all_img_sorted_ids = torch.sort(all_img_cos_sim_tensor, descending=True)[1]
print(torch.nonzero(all_img_sorted_ids == 0))
print(all_img_cos_sim_tensor[0])

In [ ]:
print(torch.max(all_patch_cos_sim_ls2[0]), torch.argmax(all_patch_cos_sim_ls2[0]))
print(torch.sort(all_patch_cos_sim_ls2[0], descending=True))

In [ ]:
print(list(masks[0].keys()))
print(np.sum(masks[0]['segmentation']), masks[0]["area"])

In [ ]:
import cv2
def generate_dialated_mask(mask, dilation_size=5):
    kernel = np.ones((dilation_size, dilation_size), np.uint8)

    binary_image = np.array(mask).astype(np.uint8)

    # Perform dilation
    dilated_image = cv2.dilate(binary_image, kernel, iterations=1)
    # from skimage.morphology import disk, dilation
    # selem = disk(dilation_size)
    # return dilation(mask, selem)
    return dilated_image


def generate_dialated_mask_ls(segmentations, dilation_size=10):
    return [generate_dialated_mask(segment['segmentation'], dilation_size) for segment in segmentations]


dilated_masks = generate_dialated_mask_ls(masks)

In [ ]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)
    
import cv2
image = np.array(raw_img_ls[0])
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


plt.figure(figsize=(20,20))
masked_image = np.copy(raw_img_ls[0])
# masked_image[np.logical_and(~masks[2]['segmentation'], ~masks[1]['segmentation'])] = 255
# masked_image[~masks[4]['segmentation']] = 255
plt.imshow(masked_image)
# show_anns(masks)
plt.axis('off')
plt.show()

In [ ]:
import cv2
def are_segments_neighbors(seg1_mask, seg2_mask):
    """Check if two image segments are neighbors using connected component labeling."""
    # Find contours for the segments
    # seg1_mask = Image.fromarray(seg1_mask)
    # seg2_mask = Image.fromarray(seg2_mask)
    
    contours1, _ = cv2.findContours(seg1_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours2, _ = cv2.findContours(seg2_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Check if any contour from one segment intersects with any contour from the other segment
    for contour1 in contours1:
        for contour2 in contours2:
            intersection = cv2.pointPolygonTest(contour2, tuple(contour1[0][0]), measureDist=False)
            if intersection == 0 or intersection == 1:
                return True  # Contours intersect, segments are neighbors
    return False

def are_segments_neighbors0(seg1_mask, seg2_mask):
    intersection = np.logical_and(seg1_mask, seg2_mask)
    return np.any(intersection)


# is_neighbor = are_segments_neighbors(masks[1]['segmentation'].astype(np.uint8), masks[2]['segmentation'].astype(np.uint8))
is_neighbor = are_segments_neighbors0(dilated_masks[1], dilated_masks[2])
print(is_neighbor)

In [ ]:
gray = cv2.cvtColor(np.array(raw_img_ls[0]),cv2.COLOR_BGR2GRAY)
_,binary = cv2.threshold(gray,100,255,cv2.THRESH_BINARY)

image,contours = cv2.findContours(binary,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
print(binary.dtype)
print(masks[1]['segmentation'])

In [ ]:
def derive_all_neighbors(mask_ls):

    neighbor_mapping_ls = []
    for i in range(len(mask_ls)):
        curr_neighbors = []
        for j in range(len(mask_ls)):
            if j != i:
                # is_neighbor = are_segments_neighbors(masks[i]['segmentation'], masks[j]['segmentation'])
                is_neighbor = are_segments_neighbors0(mask_ls[i], mask_ls[j])
                if is_neighbor:
                    curr_neighbors.append(j)
        neighbor_mapping_ls.append(curr_neighbors)
    return neighbor_mapping_ls

neighbor_mapping_ls = derive_all_neighbors(dilated_masks)
print(neighbor_mapping_ls[4])

In [ ]:
all_mask = np.zeros(masks[0]['segmentation'].shape).astype(bool)
for idx in neighbor_mapping_ls[4]:
    print(idx)
    all_mask = np.logical_or(all_mask, masks[idx]['segmentation'])

all_mask = np.logical_or(all_mask, masks[4]['segmentation'])

In [ ]:
print(masks[4])

In [ ]:
kernel = np.ones((3, 3), np.uint8)

binary_image = np.array(masks[1]['segmentation']).astype(np.uint8)

# Perform dilation
dilated_image = cv2.dilate(binary_image, kernel, iterations=1)

In [ ]:
print(np.sum(binary_image))
print(np.unique(dilated_image))

In [ ]:
raw_img_ls[0].size

In [ ]:
# tree:0, 10
# car:13
import ipyplot
img_id=0
# fig, axes = plt.subplots(10, 1, figsize=(12, 8))

def show_all_masked_image_ls(raw_img, masks):

        masked_image_ls = []

        # for i, ax in enumerate(axes.flat):
        for i in range(len(masks)):
                # if i < 10:
                masked_image = np.copy(raw_img)
                masked_image[~masks[i]['segmentation']] = 255
                masked_image_ls.append(masked_image)
                # masked_image = Image.fromarray(masked_image)
                # ax.imshow(masked_image.resize(12, 8))
                # ax.axis("off")

        ipyplot.plot_images(masked_image_ls, max_images=len(masks))   

show_all_masked_image_ls(raw_img_ls[img_id], masks_ls[img_id])
        # ax.set_title(f"Segment {i}")
# plt.show()
        # else:
        #     ax.axis("off")


# plt.figure(figsize=(20,20))
# for idx in range(len(masks)):
#     masked_image = np.copy(raw_img_ls[0])
#     masked_image[~masks[idx]['segmentation']] = 255
# # masked_image[~all_mask.astype(bool)] = 255
#     plt.imshow(masked_image)
#     plt.title(f"Segment {idx}")
# # show_anns(masks)
#     plt.axis('off')
# plt.show()

In [ ]:
masks = masks_ls[img_id]
all_mask = np.zeros(masks[0]['segmentation'].shape).astype(bool)
# mask_id_ls = [2,29,130]
# mask_id_ls = range(8,9)
# mask_id_ls=[8, 12, 20, 170]#, 173, 149, 134, 2, 3, 44, 5, 6, 9, 8, 15] + list(range(1,100))
# mask_id_ls=[20, 12]
# mask_id_ls = [3,  12,   0,   6]
mask_id_ls = [3,   12]
# mask_id_ls=[65,  17,  37, 112]
# mask_id_ls=[65]
# mask_id_ls=[1]

# mask_id_ls = [0,8]
# for idx in range(len(masks)):
for idx in mask_id_ls:
    # print(idx)
    all_mask = np.logical_or(all_mask, masks[idx]['segmentation'])

plt.figure(figsize=(20,20))
# # for idx in range(len(masks)):


# masked_image = np.copy(raw_img_ls[img_id])
# masked_image[~all_mask] = 255
# masked_image = Image.fromarray(masked_image)

# y_ls, x_ls = np.nonzero(all_mask)
# bbox = [np.min(x_ls), np.min(y_ls), np.max(x_ls), np.max(y_ls)]

# curr_patch = PIL.Image.new('RGB', raw_img_ls[img_id].size)
# curr_patch.paste(masked_image.copy().crop(bbox), box=bbox)
# masked_image0 = curr_patch # PIL.ImageOps.pad(curr_patch.crop(bbox), (224, 224))
# masked_image = PIL.ImageOps.pad(curr_patch.crop(bbox), curr_patch.size)



y_ls, x_ls = np.nonzero(all_mask)
bbox = [np.min(x_ls), np.min(y_ls), np.max(x_ls), np.max(y_ls)]
curr_patch = PIL.Image.new('RGB', raw_img_ls[img_id].size)
curr_patch.paste(raw_img_ls[img_id].copy().crop(bbox), box=bbox)
masked_image0 = curr_patch # PIL.ImageOps.pad(curr_patch.crop(bbox), (224, 224))
# masked_image = PIL.ImageOps.pad(curr_patch.crop(bbox), curr_patch.size)
masked_image = curr_patch.crop(bbox)#.resize((224, 224))
# masked_image[~all_mask.astype(bool)] = 255
plt.imshow(masked_image)
plt.title(f"Segment {idx}")
# show_anns(masks)
plt.axis('off')
plt.show()

In [ ]:
masked_image_emb = get_image_embeddings([masked_image], cl.input_processor, cl.input_to_latent, cl.model, not_normalize=False)

In [ ]:
cosin_sim4 = torch.nn.functional.cosine_similarity(text_features, masked_image_emb.view(1,-1).to(device), dim=-1)
print(cosin_sim4)

In [ ]:
cosin_sim4 = torch.nn.functional.cosine_similarity(text_features, img_emb[15].view(1,-1).to(device), dim=-1)
print(cosin_sim4)